In [8]:
pip install grpc -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [6 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\HUMENGYUAN\AppData\Local\Temp\pip-install-ibtj0j1i\grpc_cdb603d7633f4479a017809dd287530d\setup.py", line 33, in <module>
      raise RuntimeError(HINT)
  RuntimeError: Please install the official package with: pip install grpcio
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
import os, gc, warnings, random, pickle
from pathlib import Path #文件系统路径的对象化处理，比直接用字符串更安全、方便的路径操作。
import numpy as np
import pandas as pd
import polars as pl
from tqdm.auto import tqdm
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
# import torch

In [6]:
class Config:

    VERSION=2
    SEED=42
    N_FOLDS=53
    BOOSTERS=['lgbm','xgb','cat']
    MAX_ROUNDS=2500
    EARLY_STOPP=100
    VERBOSE=1
    Data_dir=r'./'
    MODEL_DIR=Path('./models');os.makedirs(Path('./models'),exist_ok=True) # exist_ok =True则不抛出异常
    OOF_DIR=Path('./oof');os.makedirs(Path('./oof'),exist_ok=True)
    TARGET_COUNT=424
    FEATURES_TO_ADD=['target_id']
    
    XGB_PARAMS={'objective': 'regression','metric': 'rmse'
        , 'learning_rate': 0.005,'max_depth':4,'random_state':SEED
        ,'tree_method':'hist','predictor':'predictor'}

In [7]:
def add_features(df):
    df['dayofweek'] = df['date_id'] % 7
    df['month'] = (df['date_id'] // 30) % 12
    df['quarter'] = df['month'] // 3
    df['day_of_month'] = df['date_id'] % 30

    df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)
    df['is_month_start'] = (df['day_of_month'] == 0).astype(int)
    df['is_month_end'] = (df['day_of_month'] == 29).astype(int)

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0, inplace=True)
    return df

In [8]:
train=pl.read_csv(Config.Data_dir+r'/train.csv')
test=pl.read_csv('test.csv')
train_labels=pl.read_csv('train_labels.csv')

In [9]:
train.shape,train_labels.shape,test.shape

((1917, 558), (1917, 425), (90, 559))

In [11]:
train_labels

date_id,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_13,target_14,target_15,target_16,target_17,target_18,target_19,target_20,target_21,target_22,target_23,target_24,target_25,target_26,target_27,target_28,target_29,target_30,target_31,target_32,target_33,target_34,target_35,…,target_387,target_388,target_389,target_390,target_391,target_392,target_393,target_394,target_395,target_396,target_397,target_398,target_399,target_400,target_401,target_402,target_403,target_404,target_405,target_406,target_407,target_408,target_409,target_410,target_411,target_412,target_413,target_414,target_415,target_416,target_417,target_418,target_419,target_420,target_421,target_422,target_423
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0.005948,-0.002851,-0.004675,-0.000639,null,null,-0.006729,0.006066,null,0.003446,null,-0.005674,null,0.000306,null,-0.005366,-0.004691,null,0.004416,null,0.008382,0.006122,null,-0.000908,null,0.000863,0.001459,0.00135,-0.013582,-0.005273,0.000193,0.005351,-0.003309,null,null,null,…,null,-0.011773,0.023571,null,-0.091022,0.006293,0.022177,0.040977,0.002379,-0.051436,0.005262,null,-0.066416,0.007124,-0.014872,null,null,null,0.035734,0.02009,null,-0.042561,-0.012987,0.027634,-0.041252,0.031637,null,null,0.021239,-0.005595,null,-0.004628,0.033793,null,0.038234,null,0.02731
1,0.005783,-0.024118,-0.007052,-0.018955,-0.031852,-0.019452,0.003002,-0.006876,-0.002042,0.021284,0.017698,0.004791,0.010499,-0.01831,0.013664,0.023292,0.007293,-0.00872,-0.021583,0.008823,-0.003377,-0.020721,-0.022573,0.012265,-0.01707,-0.026115,0.01137,-0.000112,0.000741,0.028171,-0.003402,-0.001346,-0.010453,-0.002559,-0.022646,-0.0017,…,0.004824,-0.010374,0.006768,-0.009518,-0.070225,-0.001248,0.059259,0.020991,0.008073,-0.040249,-0.001909,0.00972,-0.051488,-0.007198,-0.012201,-0.028174,0.028013,-0.037905,0.019292,0.017704,-0.018676,-0.022584,-0.005958,0.021191,-0.040295,0.029351,-0.006528,0.003377,0.021372,-0.001517,0.012846,0.010547,0.030527,-0.000764,0.025021,0.003548,0.02094
2,0.001048,0.023836,-0.008934,-0.02206,null,null,0.037449,0.007658,null,-0.026844,null,-0.0021,null,0.029436,null,0.010725,-0.00848,null,-0.004744,null,0.009616,-0.008239,null,0.001071,null,-0.012131,-0.006961,0.000797,-0.004581,0.01177,0.033271,0.009655,0.014027,null,null,null,…,0.041938,0.005172,-0.000601,-0.031099,-0.056066,-0.000063,0.115478,0.036307,0.015766,-0.052541,0.010377,0.020045,-0.045181,-0.012679,0.003707,-0.011758,0.038694,-0.037981,0.005764,-0.001951,-0.012845,-0.007365,0.008149,0.013421,-0.090242,0.016779,-0.003223,-0.006712,0.009308,0.001857,-0.012761,-0.002345,0.017529,-0.005394,0.004835,-0.009075,0.001706
3,0.0017,-0.024618,0.011943,0.004778,null,null,-0.012519,-0.016896,null,0.014843,null,0.004492,null,-0.03279,null,0.000497,0.017481,null,-0.013823,null,-0.017004,-0.002941,null,-0.003079,null,-0.011266,0.003202,0.012525,0.009794,0.00569,0.005965,-0.010537,0.030793,null,null,null,…,null,-0.014942,0.019746,null,-0.007621,0.009981,0.087343,0.029563,-0.00951,-0.017441,0.03651,null,-0.04042,0.001292,-0.016785,null,null,null,-0.01745,0.015516,null,0.028805,-0.015701,0.00144,-0.062346,0.068154,null,null,0.03688,-0.015189,null,0.008118,0.001079,null,-0.015102,null,-0.03301
4,-0.003272,0.005234,0.006856,0.013312,0.023953,0.010681,-0.011649,0.002019,0.003897,-0.008985,-0.010695,-0.009609,0.000433,0.015363,-0.007354,-0.019111,-0.000852,0.013883,0.020858,0.0013,0.001823,0.014236,0.022863,-0.020243,0.018963,0.016213,-0.026047,0.004114,0.006094,-0.018283,-0.017707,-0.002103,-0.030406,-0.005509,0.01967,0.005943,…,null,null,null,null,null,-0.00815,null,null,null,null,null,null,null,-0.005175,null,-0.00889,null,null,-0.029838,-0.03

date_id,LME_AH_Close,LME_CA_Close,LME_PB_Close,LME_ZS_Close,JPX_Gold_Mini_Futures_Open,JPX_Gold_Rolling-Spot_Futures_Open,JPX_Gold_Standard_Futures_Open,JPX_Platinum_Mini_Futures_Open,JPX_Platinum_Standard_Futures_Open,JPX_RSS3_Rubber_Futures_Open,JPX_Gold_Mini_Futures_High,JPX_Gold_Rolling-Spot_Futures_High,JPX_Gold_Standard_Futures_High,JPX_Platinum_Mini_Futures_High,JPX_Platinum_Standard_Futures_High,JPX_RSS3_Rubber_Futures_High,JPX_Gold_Mini_Futures_Low,JPX_Gold_Rolling-Spot_Futures_Low,JPX_Gold_Standard_Futures_Low,JPX_Platinum_Mini_Futures_Low,JPX_Platinum_Standard_Futures_Low,JPX_RSS3_Rubber_Futures_Low,JPX_Gold_Mini_Futures_Close,JPX_Gold_Rolling-Spot_Futures_Close,JPX_Gold_Standard_Futures_Close,JPX_Platinum_Mini_Futures_Close,JPX_Platinum_Standard_Futures_Close,JPX_RSS3_Rubber_Futures_Close,JPX_Gold_Mini_Futures_Volume,JPX_Gold_Rolling-Spot_Futures_Volume,JPX_Gold_Standard_Futures_Volume,JPX_Platinum_Mini_Futures_Volume,JPX_Platinum_Standard_Futures_Volume,JPX_RSS3_Rubber_Futures_Volume,JPX_Gold_Mini_Futures_settlement_price,JPX_Gold_Rolling-Spot_Futures_settlement_price,…,FX_AUDUSD,FX_CADJPY,FX_CHFJPY,FX_EURAUD,FX_EURGBP,FX_EURJPY,FX_EURUSD,FX_GBPAUD,FX_GBPJPY,FX_GBPUSD,FX_NZDJPY,FX_NZDUSD,FX_USDCHF,FX_USDJPY,FX_ZARJPY,FX_ZARUSD,FX_NOKUSD,FX_NOKEUR,FX_CADUSD,FX_AUDNZD,FX_EURCHF,FX_EURCAD,FX_AUDCAD,FX_GBPCHF,FX_EURNZD,FX_AUDCHF,FX_GBPNZD,FX_GBPCAD,FX_CADCHF,FX_NZDCAD,FX_NZDCHF,FX_ZAREUR,FX_NOKGBP,FX_NOKCHF,FX_ZARCHF,FX_NOKJPY,FX_ZARGBP
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,2264.5,7205.0,2570.0,3349.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.783393,89.764583,115.545884,1.540368,0.887622,135.44991,1.206713,1.735387,152.598635,1.35949,79.721293,0.710231,0.97145,112.247002,9.028141,0.080431,0.123146,0.102051,0.799706,1.103011,1.172261,1.508946,0.979601,1.320676,1.699043,0.761027,1.914152,1.699987,0.776874,0.888115,0.689954,0.066653,0.090582,0.11963,0.078135,13.82274,0.059163
1,2228.0,7147.0,2579.0,3327.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.782779,89.731507,115.23516,1.534906,0.889248,135.272384,1.201492,1.726072,152.119947,1.351132,79.815128,0.70892,0.977019,112.586998,9.111194,0.080926,0.123355,0.102668,0.796997,1.104186,1.173881,1.507524,0.98216,1.320083,1.694821,0.76479,1.905904,1.695279,0.778682,0.889488,0.692628,0.067354,0.091297,0.12052,0.079066,13.888146,0.059895
2,2250.0,7188.5,2587.0,3362.0,4684.0,4691.0,4684.0,3363.0,3367.0,207.0,4735.0,4746.0,4735.0,3443.0,3449.0,207.7,4679.0,4688.0,4679.0,3362.0,3362.0,206.4,4727.0,4739.0,4730.0,3426.0,3427.0,206.9,2681.0,37908.0,30656.0,624.0,13713.0,4128.0,4730.0,4735.0,…,0.786472,90.306741,115.750255,1.535084,0.89083,136.176212,1.2073,1.723206,152.864365,1.355253,80.734616,0.71577,0.97446,112.793999,9.177484,0.081365,0.123975,0.102688,0.800634,1.098776,1.176466,1.50793,0.982311,1.32064,1.686714,0.766385,1.893418,1.692724,0.780186,0.894004,0.69749,0.067394,0.091478,0.120809,0.079287,13.983675,0.060037
3,2202.5,7121.0,2540.0,3354.0,4728.0,4737.0,4729.0,3430.0,3426.0,207.0,4795.0,4797.0,4789.0,3493.0,3494.0,207.6,4727.0,4734.0,4726.0,3430.0,3425.0,204.0,4778.0,4788.0,4780.0,3486.0,3486.0,204.8,3523.0,30791.0,35267.0,525.0,17629.0,4586.0,4780.0,4787.0,…,0.787461,91.079905,115.976719,1.527972,0.886809,135.945666,1.203219,1.723001,153.297609,1.356796,81.010025,0.716998,0.974204,112.985001,9.195264,0.081385,0.124225,0.103244,0.806124,1.098275,1.172181,1.492598,0.976849,1.321796,1.678134,0.767148,1.892329,1.683111,0.785329,0.889439,0.698502,0.067639,0.091558,0.121021,0.079285,1